In [1]:
import sys
import os

sys.path.append('..')

import utilities.functions as functions

from utilities.functions import (
    load_data,
    check_key_uniqueness,
    save_parquet,
    merge_df,
    load_orders,
    process_orders_pandas,
)

Load the data --- se necessario salvar em stage - neste momento o estara comentado

In [2]:
URL_CONSUMER = "https://data-architect-test-source.s3-sa-east-1.amazonaws.com/consumer.csv.gz"
df_consumer = load_data(URL_CONSUMER)[["customer_id", "active","created_at"]]

In [3]:
URL_RESTAURANT ="https://data-architect-test-source.s3-sa-east-1.amazonaws.com/restaurant.csv.gz"
df_restaurant= load_data(URL_RESTAURANT)

In [4]:
ab_test_url = "https://data-architect-test-source.s3-sa-east-1.amazonaws.com/ab_test_ref.tar.gz"
df_ab = load_data(ab_test_url)

In [5]:
#save_parquet(df_consumer, "stage", "df_consumer.parquet")
#save_parquet(df_restaurant, "stage", "df_restaurant.parquet")
#save_parquet(df_consumer, "stage", "df_consumer.parquet")

Bronze layer - verify duplicates e nulo. e se necessario remover

In [6]:
check_key_uniqueness(df_consumer, ["customer_id","active","created_at"])

✅ Colunas ['customer_id', 'active', 'created_at'] são NOT NULL e UNIQUE.


(True, None, None, None)

In [7]:
check_key_uniqueness(df_restaurant, ["id"])

✅ Colunas ['id'] são NOT NULL e UNIQUE.


(True, None, None, None)

In [8]:
check_key_uniqueness(df_ab, ["customer_id","is_target"])
print(df_ab[df_ab["customer_id"].isna()])


❌ Colunas ['customer_id', 'is_target'] contêm valores nulos.

Soma de nulos por coluna:
customer_id    1
is_target      0
dtype: int64

Índices com nulos:
[81149]
      customer_id is_target
81149         NaN    target


In [9]:
df_ab_np = df_ab[~df_ab["customer_id"].isna()]

Antes de carregar a base ordens sera definido o publico todal, e da base de ordem serao filtrados somentes os clientes elegiceis

In [10]:
df_publico=merge_df(df_ab_np,df_consumer,['customer_id'],'outer')

In [11]:
clientes_mes = (
    df_publico.groupby(['active', 'is_target'], dropna=False)
              ['customer_id']
              .nunique()
              .reset_index(name='numero_clientes_distintos')
)
clientes_mes

,active,is_target,numero_clientes_distintos
0,False,control,713
1,False,target,882
2,True,control,359700
3,True,target,444861
4,NaN,control,129
5,NaN,target,181


In [12]:
df_publico = df_publico.dropna(subset=['active'])

In [13]:
clientes_mes = (
    df_publico.groupby(['active', 'is_target'], dropna=False)
              ['customer_id']
              .nunique()
              .reset_index(name='numero_clientes_distintos')
)
clientes_mes

,active,is_target,numero_clientes_distintos
0,False,control,713
1,False,target,882
2,True,control,359700
3,True,target,444861


Publico definido e todos os clientes marcados no teste a/b e existentes na base de clientes

In [14]:
save_parquet(df_publico, "silver", "df_publico.parquet")

💾 SALVO: dados/silver/df_publico.parquet
📁 Local: /Users/maceli/ifood_case/code/notebook/dados/silver/df_publico.parquet


PosixPath('dados/silver/df_publico.parquet')

Da base de ordens serao filtrados todos os clientes com orden nos meses de de dezembro e janeiro

In [15]:
#df_ab=df_ab.head(5000)

In [ ]:
URL_ORDERS = "https://data-architect-test-source.s3-sa-east-1.amazonaws.com/order.json.gz"

COLUMNS_TO_DROP = [
    'cpf','customer_name','delivery_address_city','delivery_address_country',
    'delivery_address_district','delivery_address_external_id',
    'delivery_address_latitude','delivery_address_longitude',
    'delivery_address_state','delivery_address_zip_code','items',
    'merchant_latitude','merchant_longitude','merchant_timezone',
    'order_scheduled','order_scheduled_date'
]

customer_ids = df_ab["customer_id"].astype(str).unique()

df_orders = load_orders(
    url=URL_ORDERS,
    customer_ids=customer_ids,
    columns_to_drop=COLUMNS_TO_DROP
)

df_orders.head()


In [ ]:
print(check_key_uniqueness(df_orders, ["customer_id","merchant_id","order_id"]))
print(check_key_uniqueness(df_orders, ["customer_id","merchant_id","order_id","order_created_at"]))

❌ Colunas ['customer_id', 'merchant_id', 'order_id'] possuem duplicações.
(False,                                              customer_id  \
9705   c4de9a5357a04d1dc0418c454dac3a427989dc2a0abcfd...   
9706   e1b7be7dd8e881a5eb6f3dc6d3d68a3d81017852941671...   
9707   4ff86769b4a3ad71d5640bd43c8e3d4e968d87c45dc730...   
9708   bcc7bd4f084a6c1dde231fa8d6c026be32b47b3caad944...   
9709   9ea583e48192ef77a720628abdd603bfcb67e8875fadac...   
...                                                  ...   
29716  e48cc11ea8ac7975fba29816691d0a6fa2373166d21953...   
29717  ff41a05fd3eb764706447c4b4b5fea67c09347357b0cdf...   
29718  6021262abe263e6db68debe0cf04ba68a797ecde025500...   
29719  2fa27d78d5b5e830fe1ac06f6895c48fee1cdb9cb3fe0c...   
29720  d1dbb74f136e132e414acca30bbb738dcc0b46fd4434b1...   

                                             merchant_id  \
9705   a28f3af17e31644f404358d7c4a5cf06d54a715b859882...   
9706   352707fb0b8e87d5d4b1dbdf402a608eb7a369a8ce46a4...   
9707   1c1b260dcb

In [ ]:
#save_parquet(df_orders, "stage", "df_orders.parquet")

Add orders para a base de publico

In [ ]:
df_publico_orders=merge_df(df_publico,df_orders,['customer_id'],'inner')

In [ ]:
df_publico_orders.head()

,customer_id,is_target,active,created_at,merchant_id,order_created_at,order_id,order_total_amount,origin_platform
0,0004d845ebbec6067049d15c5c8c49ab7553e13eeeb901...,target,True,2018-01-08T13:31:39.089Z,b5f4d43a97a1ca81c89af74bdd5d88273081506b10a440...,2019-01-11T23:44:14.000Z,1148f9b888260237c9407e564c8db012e7f332bd44e17e...,136.50,IOS
1,0004d845ebbec6067049d15c5c8c49ab7553e13eeeb901...,target,True,2018-01-08T13:31:39.089Z,b5f4d43a97a1ca81c89af74bdd5d88273081506b10a440...,2018-12-12T23:44:14.000Z,1148f9b888260237c9407e564c8db012e7f332bd44e17e...,136.50,IOS
2,0018bea16d2204ce6490e8ef3f964167e7be1386180924...,target,True,2018-01-06T23:21:28.264Z,0298247e3d5a8ea6ee57b5716cb36f62ec4ca62da1a5a8...,2019-01-25T23:05:55.000Z,e2723167c286616dac99cd46119aac60c56d4288db3c5b...,51.00,ANDROID
3,0018bea16d2204ce6490e8ef3f964167e7be1386180924...,target,True,2018-01-06T23:21:28.264Z,c8c68b7378452469dc3d0fd0aab92ee12416d7b73cee95...,2019-01-28T22:59:14.000Z,b749053fcda42ca90b13b5db866caab3407f5ae3668dce...,55.96,ANDROID
4,0018bea16d2204ce6490e8ef3f964167e7be1386180924...,target,True,2018-01-06T23:21:28.264Z,ebd86c209e5c1805eaa302edd9370697a82d18485f3884...,2019-01-01T14:56:21.000Z,d9f725db283b815e89d457d03f80a3e08dc34d4277ea49...,96.60,ANDROID


Construcao de chave unica, e sumarizacoes visao cliente

In [ ]:
#df_publico_orders=df_publico_orders.head(1000)

In [ ]:
df_publico=process_orders_pandas(df_publico_orders)

In [ ]:
save_parquet(df_publico, "silver", "df_publico.parquet")

💾 SALVO: dados/silver/df_publico.parquet
📁 Local: /Users/maceli/ifood_case/code/notebook/dados/silver/df_publico.parquet


PosixPath('dados/silver/df_publico.parquet')

Uma linha por cliente, com as variaveis necessarias

In [ ]:
df_pub_un = df_publico[["customer_id",'origin_platform',"is_target", "order_created_month", "num_pedidos_mes", "num_pedidos_hist",'total_amount_mes','ticket_medio']].drop_duplicates().reset_index(drop=True)
df_pub_un.head()

,customer_id,origin_platform,is_target,order_created_month,num_pedidos_mes,num_pedidos_hist,total_amount_mes,ticket_medio
0,fff0bcf899914ef1a4441277ca547b0282750344e95090...,ANDROID,target,12,3,9,160.00,53.333333
1,fff0bcf899914ef1a4441277ca547b0282750344e95090...,ANDROID,target,1,6,9,260.99,43.498333
2,fff02e65f78e3377668f6d87e6a7ff8f0ee0f18e4fd6a5...,IOS,target,12,4,12,152.50,38.125000
3,fff02e65f78e3377668f6d87e6a7ff8f0ee0f18e4fd6a5...,IOS,target,1,8,12,326.40,40.800000
4,ffee1b5d2c53a4808115bbfd35c296349e42dad9f94d19...,ANDROID,control,12,2,5,164.09,82.045000


Salvar base visao cliente em gold layer

In [ ]:
save_parquet(df_pub_un, "gold", "df_pub_un.parquet")

💾 SALVO: dados/gold/df_pub_un.parquet
📁 Local: /Users/maceli/ifood_case/code/notebook/dados/gold/df_pub_un.parquet


PosixPath('dados/gold/df_pub_un.parquet')